## Setup

In [ ]:
from specific import *

### Get shifted data

In [ ]:
(
    endog_data,
    exog_data,
    master_mask,
    filled_datasets,
    masked_datasets,
    land_mask,
) = get_offset_data()

### Retrieve previous results from the 'model' notebook

In [ ]:
X_train, X_val, y_train, y_val = data_split_cache.load()
rf = get_model()
masked_train_data = get_mm_data(y_train.values, master_mask, "train")
masked_val_data = get_mm_data(y_val.values, master_mask, "val")

### Predict out-of-sample BA

In [ ]:
n_threads = get_ncpus()
rf.n_jobs = n_threads
with parallel_backend("threading", n_jobs=n_threads):
    predicted_ba = get_mm_data(rf.predict(X_val), master_mask, "val")

In [ ]:
model_name = "RF1"

# Plotting params.


def get_plot_kwargs(cbar_label="Burned Area Fraction", **kwargs):
    return {
        **dict(
            fig=plt.figure(figsize=(5.1, 2.8)),
            colorbar_kwargs={"label": cbar_label, "format": "%0.0e"},
            coastline_kwargs={"linewidth": 0.3},
            cmap="brewer_RdYlBu_11",
        ),
        **kwargs,
    }


date_str = "2010-01 to 2015-01"

In [ ]:
assert np.all(predicted_ba.mask == masked_val_data.mask)

In [ ]:
boundaries = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]

cmap = "YlOrRd"
extend = "both"

# Plotting predicted.
fig = cube_plotting(
    predicted_ba,
    **get_plot_kwargs(
        cmap=cmap,
        title=f"Predicted BA\n{date_str}",
        boundaries=boundaries,
        extend=extend,
    ),
)
figure_saver.save_figure(fig, f"predicted_ba_{model_name}", sub_directory="predictions")

# Plotting observed.
fig = cube_plotting(
    masked_val_data,
    **get_plot_kwargs(
        cmap=cmap,
        title=f"Observed BA\n{date_str}",
        boundaries=boundaries,
        extend=extend,
    ),
)
figure_saver.save_figure(fig, f"observed_ba_{model_name}", sub_directory="predictions")

In [ ]:
frac_diffs = (masked_val_data - predicted_ba) / masked_val_data

In [ ]:
# Plotting differences.
diff_boundaries = [-1e1, -1e0, 0, 1e-1]
extend = "both"

fig = cube_plotting(
    frac_diffs,
    **get_plot_kwargs(
        title=f"BA Discrepancy <(Obs. - Pred.) / Obs.> \n{date_str}",
        cmap_midpoint=0,
        boundaries=diff_boundaries,
        colorbar_kwargs={"label": "(Obs. - Pred.) / Obs.", "format": "%0.0e"},
        extend=extend,
    ),
)
figure_saver.save_figure(
    fig, f"difference_ba_{model_name}", sub_directory="predictions"
)

In [ ]:
rel_mse = frac_diffs ** 2

In [ ]:
# Plotting differences.
diff_boundaries = [1e-1, 1, 1e1, 1e2, 1e3]
extend = "both"

fig = cube_plotting(
    rel_mse,
    **get_plot_kwargs(
        cmap="inferno",
        title=r"BA Discrepancy <$\mathrm{((Obs. - Pred.) / Obs.)}^2$>"
        + f"\n{date_str}",
        boundaries=diff_boundaries,
        colorbar_kwargs={"label": "1", "format": "%0.0e"},
        extend=extend,
    ),
)
figure_saver.save_figure(
    fig, f"rel_mse_difference_ba_{model_name}", sub_directory="predictions"
)